In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import joblib
import random
import math
import zlib
from collections import Counter
from scipy.stats import entropy
from google.colab import drive
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Dropout, Input

drive.mount('/content/drive')

BASE_PATH = '/content/drive/MyDrive/Bioinformatica'
DATASET_PATH = f'{BASE_PATH}/Dataset/complete_fusion_dataset.csv'
MODELS_ROOT = f'{BASE_PATH}/Modelli'

WINDOW_HALF = 500
TOTAL_LEN = WINDOW_HALF * 2
if not os.path.exists(MODELS_ROOT): os.makedirs(MODELS_ROOT)

def load_and_clean_data():
    df = pd.read_csv(DATASET_PATH)
    print(f"Righe originali: {len(df)}")

    # Rimozione duplicati con tutte le colonne in comune
    subset_cols = ['sequence', 'gene1', 'gene2','junction_point']
    df = df.drop_duplicates(subset=subset_cols).copy()
    print(f"Righe dopo pulizia duplicati: {len(df)}")

    return df

def get_smart_window_robust(row):
    """
    Versione Robust: Non scarta quasi nulla.
    - Se < 1000bp: Restituisce tutto.
    - Se > 1000bp: Taglia 1000bp gestendo i bordi per non uscire fuori.
    """
    seq = row['sequence']
    slen = len(seq)

    if slen <= TOTAL_LEN:
        return seq

    if row['label'] == 1:
        try:
            pivot = int(row['junction_point'])
        except:
            pivot = slen // 2
    else:
        random.seed(slen)
        min_p = WINDOW_HALF
        max_p = slen - WINDOW_HALF
        pivot = random.randint(min_p, max_p)

    start = pivot - WINDOW_HALF
    end = pivot + WINDOW_HALF

    if start < 0:
        start = 0
        end = TOTAL_LEN

    if end > slen:
        end = slen
        start = slen - TOTAL_LEN

    if start < 0: start = 0
    if end > slen: end = slen

    return seq[start:end]

df_main = load_and_clean_data()
df_main['window'] = df_main.apply(get_smart_window_robust, axis=1)
df_main = df_main.dropna(subset=['window']).copy()

print(f"Dataset Recuperato: {len(df_main)} campioni.")

def train_and_evaluate_group(X, y, group_name, version_id):
    save_path = f"{MODELS_ROOT}/{version_id}_{group_name}"
    if not os.path.exists(save_path): os.makedirs(save_path)

    # Split & Scaling
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train)
    X_test_s = scaler.transform(X_test)

    joblib.dump(scaler, f"{save_path}/scaler.pkl")

    results = {}

    # 1. Random Forest
    print("Training Random Forest...")
    rf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42)
    rf.fit(X_train_s, y_train)
    joblib.dump(rf, f"{save_path}/RF_model.pkl")
    results['RF'] = rf.predict(X_test_s)

    # 2. XGBoost
    print("Training XGBoost...")
    ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)
    xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', n_jobs=-1, random_state=42, scale_pos_weight=ratio)
    xgb.fit(X_train_s, y_train)
    joblib.dump(xgb, f"{save_path}/XGB_model.pkl")
    results['XGB'] = xgb.predict(X_test_s)

    # 3. MLP (Multi-Layer Perceptron)
    print("Training MLP...")

    model_mlp = Sequential([
        Input(shape=(X_train_s.shape[1],)),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    model_mlp.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
    model_mlp.fit(X_train_s, y_train, epochs=20, batch_size=32, verbose=0, callbacks=[callback])

    model_mlp.save(f"{save_path}/MLP_model.keras")
    preds_mlp = (model_mlp.predict(X_test_s) > 0.5).astype("int32").flatten()
    results['MLP'] = preds_mlp

    # Report
    print(f"\nResults: {version_id} - {group_name}")
    for name, preds in results.items():
        print(f"\n--- {name} ---")
        print(f"Accuracy:  {accuracy_score(y_test, preds):.4f}")
        print(f"Precision: {precision_score(y_test, preds, zero_division=0):.4f}")
        print(f"Recall:    {recall_score(y_test, preds, zero_division=0):.4f}")
        print(f"F1-Score:  {f1_score(y_test, preds, zero_division=0):.4f}")
        print("Confusion Matrix:")
        print(confusion_matrix(y_test, preds))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Righe originali: 265080
Righe dopo pulizia duplicati: 230425
Dataset Recuperato: 230425 campioni.


In [ ]:
# Gruppo 1: Statistiche di base
# Feature: GC Content, Shannon Entropy (Standard)

def extract_group1(seq):
    seq = seq.upper()
    n = len(seq)
    if n == 0: return [0, 0]

    # 1. GC Content
    gc = (seq.count('G') + seq.count('C')) / n

    # 2. Shannon Entropy
    counts = Counter(seq)
    probs = [c/n for c in counts.values()]
    ent = entropy(probs, base=2)

    return [gc, ent]

print("Calcolo Feature Gruppo 1...")
X_g1 = np.array([extract_group1(s) for s in tqdm(df_main['window'])])
y_g1 = df_main['label'].values

# Addestramento
train_and_evaluate_group(X_g1, y_g1, group_name="Baseline_Stats", version_id="01")

Calcolo Feature Gruppo 1...


  0%|          | 0/230425 [00:00<?, ?it/s]

Training Random Forest...
Training XGBoost...


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [10:51:55] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Training MLP...
1441/1441 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step

Results: 01 - Baseline_Stats

--- RF ---
Accuracy:  0.7967
Precision: 0.6997
Recall:    0.7445
F1-Score:  0.7214
Confusion Matrix:
[[24582  5207]
 [ 4163 12133]]

--- XGB ---
Accuracy:  0.5917
Precision: 0.4508
Recall:    0.7084
F1-Score:  0.5510
Confusion Matrix:
[[15724 14065]
 [ 4752 11544]]

--- MLP ---
Accuracy:  0.6464
Precision: 0.0000
Recall:    0.0000
F1-Score:  0.0000
Confusion Matrix:
[[29789     0]
 [16296     0]]


In [ ]:
# GRUPPO 2.1: BWT Classica (BWT)
# Feature: BWT Entropy, Run Lengths (Mean, Max, Min), Compression Ratio -> BWT Classica

def extract_group2_1(seq):
    seq = seq.upper()
    if len(seq) == 0: return [0, 0, 0, 0, 0]

    # Calcolo BWT
    s_bwt = seq + '$'
    indices = sorted(range(len(s_bwt)), key=lambda i: s_bwt[i:])
    bwt = "".join([s_bwt[i-1] if i > 0 else s_bwt[-1] for i in indices])

    # 1. BWT Entropy
    counts = Counter(bwt)
    probs = [c/len(bwt) for c in counts.values()]
    bwt_ent = entropy(probs, base=2)

    # 2. Run Length Analysis
    runs = []
    curr = 1
    for i in range(1, len(bwt)):
        if bwt[i] == bwt[i-1]: curr += 1
        else:
            runs.append(curr)
            curr = 1
    runs.append(curr)

    mean_run = np.mean(runs) if runs else 0
    max_run = np.max(runs) if runs else 0
    min_run = np.min(runs) if runs else 0

    # 3. Compression Ratio
    comp_ratio = len(zlib.compress(bwt.encode())) / len(bwt)

    return [bwt_ent, mean_run, max_run, min_run, comp_ratio]

print("Calcolo Feature Gruppo 2...")
X_g2 = np.array([extract_group2_1(s) for s in tqdm(df_main['window'])])
y_g2 = df_main['label'].values

# Addestramento
train_and_evaluate_group(X_g2, y_g2, group_name="BWT_Classic", version_id="02")

Calcolo Feature Gruppo 2...


  0%|          | 0/230425 [00:00<?, ?it/s]

Training Random Forest...
Training XGBoost...


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [11:04:10] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Training MLP...
1441/1441 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step

Results: 02 - BWT_Classic

--- RF ---
Accuracy:  0.8415
Precision: 0.7981
Recall:    0.7387
F1-Score:  0.7672
Confusion Matrix:
[[26743  3046]
 [ 4258 12038]]

--- XGB ---
Accuracy:  0.6320
Precision: 0.4875
Recall:    0.7921
F1-Score:  0.6036
Confusion Matrix:
[[16220 13569]
 [ 3388 12908]]

--- MLP ---
Accuracy:  0.6556
Precision: 0.5324
Recall:    0.2140
F1-Score:  0.3053
Confusion Matrix:
[[26725  3064]
 [12808  3488]]


In [ ]:
# GRUPPO 2.2: POSITIONAL BWT (PBWT)
# Feature: Entropy, Run Lengths, Compression Ratio sulle distanze del Suffix Array

def extract_group2_2(seq):
    seq = seq.upper()
    n = len(seq)
    if n == 0: return [0, 0, 0, 0, 0]

    # Costruzione Suffix Array (SA)
    s_bwt = seq + '$'
    sa = sorted(range(len(s_bwt)), key=lambda i: s_bwt[i:])

    # Calcolo differenze posizionali (distanza tra indici adiacenti nel SA)
    sa_diffs = np.diff(sa)
    abs_diffs = np.abs(sa_diffs)

    # A. Positional Entropy
    counts = Counter(abs_diffs)
    probs = [c/len(abs_diffs) for c in counts.values()]
    pos_ent = entropy(probs, base=2)

    # B. Run Length Analysis (sulle distanze)
    runs = []
    curr = 1
    # Iteriamo sull'array delle differenze
    for i in range(1, len(abs_diffs)):
        if abs_diffs[i] == abs_diffs[i-1]:
            curr += 1
        else:
            runs.append(curr)
            curr = 1
    runs.append(curr)

    mean_run = np.mean(runs) if runs else 0
    max_run = np.max(runs) if runs else 0
    min_run = np.min(runs) if runs else 0

    # C. Compression Ratio del Suffix Array
    # Convertiamo l'array di interi in bytes per comprimerlo con zlib
    sa_bytes = np.array(abs_diffs, dtype=np.int32).tobytes()
    comp_ratio = len(zlib.compress(sa_bytes)) / len(sa_bytes)

    return [pos_ent, mean_run, max_run, min_run, comp_ratio]

print("Calcolo Feature Gruppo 2.2 (Positional BWT)...")
X_g2_2 = np.array([extract_group2_2(s) for s in tqdm(df_main['window'])])
y_g2_2 = df_main['label'].values

feature_names_g2_2 = [
    'PBWT_Entropy', 'PBWT_MeanRun', 'PBWT_MaxRun', 'PBWT_MinRun', 'PBWT_CompRatio'
]

# Addestramento
train_and_evaluate_group(
    X_g2_2,
    y_g2_2,
    group_name="BWT_Positional",
    version_id="02_2",
    feature_names=feature_names_g2_2
)

Calcolo Feature Gruppo 2.2 (Positional BWT)...


  0%|          | 0/230425 [00:00<?, ?it/s]

TypeError: train_and_evaluate_group() got an unexpected keyword argument 'feature_names'

In [ ]:
# GRUPPO 2.3: REVERSE-COMPLEMENT BWT

def get_reverse_complement(seq):
    # Mappa di complementarietà
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N': 'N'}
    # Lettura inversa + sostituzione
    return "".join(complement.get(base, base) for base in reversed(seq))

def extract_group2_3(seq):
    seq = seq.upper()
    if len(seq) == 0: return [0, 0, 0, 0, 0]

    # 1. Creazione Sequenza Estesa (Seq + Marker + RevComp)
    rc_seq = get_reverse_complement(seq)
    full_seq = seq + '#' + rc_seq + '$'

    # 2. Calcolo BWT sulla sequenza estesa
    indices = sorted(range(len(full_seq)), key=lambda i: full_seq[i:])
    bwt = "".join([full_seq[i-1] if i > 0 else full_seq[-1] for i in indices])

    # A. RBWT Entropy
    counts = Counter(bwt)
    probs = [c/len(bwt) for c in counts.values()]
    rbwt_ent = entropy(probs, base=2)

    # B. Run Length Analysis
    runs = []
    curr = 1
    for i in range(1, len(bwt)):
        if bwt[i] == bwt[i-1]:
            curr += 1
        else:
            runs.append(curr)
            curr = 1
    runs.append(curr)

    mean_run = np.mean(runs) if runs else 0
    max_run = np.max(runs) if runs else 0
    min_run = np.min(runs) if runs else 0

    # C. Compression Ratio
    comp_ratio = len(zlib.compress(bwt.encode())) / len(bwt)

    return [rbwt_ent, mean_run, max_run, min_run, comp_ratio]

print("Calcolo Feature Gruppo 2.3 (Reverse-Complement BWT)...")
X_g2_3 = np.array([extract_group2_3(s) for s in tqdm(df_main['window'])])
y_g2_3 = df_main['label'].values

feature_names_g2_3 = [
    'RC_Entropy', 'RC_MeanRun', 'RC_MaxRun', 'RC_MinRun', 'RC_CompRatio'
]

# Addestramento
train_and_evaluate_group(
    X_g2_3,
    y_g2_3,
    group_name="BWT_RevComp",
    version_id="02_3",
    feature_names=feature_names_g2_3
)

In [ ]:
# Gruppo 3: Analisi Bio-Linguistica
# Feature: KL Divergence, Stop Codon Ratio

def extract_group3(seq):
    seq = seq.upper()
    n = len(seq)
    if n == 0: return [0, 0]

    mid = n // 2
    left, right = seq[:mid], seq[mid:]
    k = 3

    def get_probs(s):
        kmers = [s[i:i+k] for i in range(len(s)-k+1)]
        c = Counter(kmers)
        tot = sum(c.values())
        return {k: v/tot for k,v in c.items()}

    p_l = get_probs(left)
    p_r = get_probs(right)
    all_k = set(p_l.keys()) | set(p_r.keys())

    vec_l = np.array([p_l.get(x, 1e-9) for x in all_k])
    vec_r = np.array([p_r.get(x, 1e-9) for x in all_k])

    kl = entropy(vec_l, vec_r) + entropy(vec_r, vec_l)

    # 2. Stop Codon Ratio
    stops = ['TAA', 'TAG', 'TGA']
    cnt = 0
    for frame in range(3):
        codons = [seq[i:i+3] for i in range(frame, len(seq)-2, 3)]
        for c in codons:
            if c in stops: cnt += 1
    stop_ratio = cnt / len(seq)

    return [kl, stop_ratio]

print("Calcolo Feature Gruppo 3...")
X_g3 = np.array([extract_group3(s) for s in tqdm(df_main['window'])])
y_g3 = df_main['label'].values

# Addestramento
train_and_evaluate_group(X_g3, y_g3, group_name="Bio_Linguistic", version_id="03")

In [ ]:
# Features: GC (G1) + BWT Completo (G2) + KL & Stop (G3)

print("Assemblaggio Feature Gruppo 5 (Ibrido Completo)...")

feature_gc = X_g1[:, 0].reshape(-1, 1) # Gruppo1
feature_bwt = X_g2 # Gruppo2
feature_bio = X_g3 # Gruppo3

# Gruppo Ibrido
X_g5 = np.hstack((feature_gc, feature_bwt, feature_bio))
y_g5 = y_g1

# Features in ordine di scelta dai gruppi
feature_names = [
    'GC_Content',
    'BWT_Entropy', 'BWT_MeanRun', 'BWT_MaxRun', 'BWT_MinRun', 'Compression_Ratio',
    'KL_Divergence', 'Stop_Codon'
]

print(f"Feature Matrix Shape: {X_g5.shape}")

train_and_evaluate_group(X_g5, y_g5, group_name="Hybrid_Full_Features", version_id="05")

# Analisi importanza delle features
save_path_g5 = f"{MODELS_ROOT}/05_Hybrid_Full_Features"
rf_loaded = joblib.load(f"{save_path_g5}/RF_model.pkl")

import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 6))
importances = rf_loaded.feature_importances_
indices = np.argsort(importances)[::-1]
sorted_names = [feature_names[i] for i in indices]


sns.barplot(x=sorted_names, y=importances[indices], hue=sorted_names, palette="viridis", legend=False)

plt.title("Classifica Importanza Feature (Modello Completo)")
plt.ylabel("Importanza (Gini Impurity)")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()